##  Multi-Input Models Learning

In [1]:
#A multi-model deep learning approach for image recognition

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, shutil
import cv2
from os import listdir
from os.path import isfile, join
import string
import re
import nltk
from keras import layers
from keras import Input
from keras.models import Model
from keras import models
# from tensorflow.python.keras.layers import Embedding, Dense, Input, Dropout, LSTM, Activation, Conv2D, Reshape, Average, Bidirectional
%matplotlib inline

Using TensorFlow backend.


In [2]:
import pandas as pd
dtype_dic= {'id': str}
train_df = pd.read_json(open("train.jsonl", "r", encoding="utf8"),dtype=str,lines=True)

In [3]:
train_df.head(50)

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."
5,16952,img/16952.png,0,go sports! do the thing! win the points!
6,76932,img/76932.png,0,fine you're right. now can we fucking drop it?
7,70914,img/70914.png,0,tattoos are bad for your health i know 5 milli...
8,2973,img/02973.png,0,how long can i run? till the chain tightens
9,58306,img/58306.png,0,what is he hiding? we need to see his tax retu...


In [4]:
split1 = []
for id in train_df[['img']].iterrows():
    split1.append(id[1][0].split('/'))

In [5]:
split2 = []
for data in split1:
    split2.append(data[1].split('.'))

In [6]:
split3 = []
for data in split2:
    split3.append(data[0])

In [7]:
train_df['id_new'] = split3

In [8]:
train_df.head()

,id,img,label,text,id_new
0,42953,img/42953.png,0,its their character not their color that matters,42953
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,23058
2,13894,img/13894.png,0,putting bows on your pet,13894
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403


In [9]:
train_df.pop('id')

0       42953
1       23058
2       13894
3       37408
4       82403
        ...  
8495    10423
8496    98203
8497    36947
8498    16492
8499    15937
Name: id, Length: 8500, dtype: object

In [10]:
train_df.head()

,img,label,text,id_new
0,img/42953.png,0,its their character not their color that matters,42953
1,img/23058.png,0,don't be afraid to love again everyone is not ...,23058
2,img/13894.png,0,putting bows on your pet,13894
3,img/37408.png,0,i love everything and everybody! except for sq...,37408
4,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403


In [11]:
train_df = train_df.rename(columns={'id_new': 'id'})

In [12]:
train_df['id'] = train_df['id'].astype('str')

In [13]:
train_df.head(50)

,img,label,text,id
0,img/42953.png,0,its their character not their color that matters,42953
1,img/23058.png,0,don't be afraid to love again everyone is not ...,23058
2,img/13894.png,0,putting bows on your pet,13894
3,img/37408.png,0,i love everything and everybody! except for sq...,37408
4,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403
5,img/16952.png,0,go sports! do the thing! win the points!,16952
6,img/76932.png,0,fine you're right. now can we fucking drop it?,76932
7,img/70914.png,0,tattoos are bad for your health i know 5 milli...,70914
8,img/02973.png,0,how long can i run? till the chain tightens,02973
9,img/58306.png,0,what is he hiding? we need to see his tax retu...,58306


In [14]:
import os

file_path_img = 'img/img/'
file_path_text = 'img/labels/'

files_img = [f for f in os.listdir(file_path_img) if f.endswith('.png')]
files_txt = [f for f in os.listdir(file_path_text) if f.endswith('.txt')]

In [15]:
files_img

['21753.png',
 '10582.png',
 '04615.png',
 '84316.png',
 '45109.png',
 '30642.png',
 '59231.png',
 '54397.png',
 '98047.png',
 '80614.png',
 '53296.png',
 '68721.png',
 '46895.png',
 '40216.png',
 '83921.png',
 '28160.png',
 '24769.png',
 '72940.png',
 '28976.png',
 '13902.png',
 '89302.png',
 '87023.png',
 '74059.png',
 '72893.png',
 '07463.png',
 '21650.png',
 '27856.png',
 '80941.png',
 '14965.png',
 '03942.png',
 '35680.png',
 '39854.png',
 '28134.png',
 '49271.png',
 '39256.png',
 '09347.png',
 '58419.png',
 '32674.png',
 '52691.png',
 '83429.png',
 '47290.png',
 '98305.png',
 '72654.png',
 '79568.png',
 '63418.png',
 '19372.png',
 '18306.png',
 '17493.png',
 '89503.png',
 '46387.png',
 '07261.png',
 '75160.png',
 '84016.png',
 '10867.png',
 '45269.png',
 '09128.png',
 '93610.png',
 '92016.png',
 '32915.png',
 '45827.png',
 '94617.png',
 '37298.png',
 '42681.png',
 '79162.png',
 '91768.png',
 '12678.png',
 '26598.png',
 '53976.png',
 '51973.png',
 '50462.png',
 '04397.png',
 '9702

In [16]:
files_txt

['36850.txt',
 '54028.txt',
 '04729.txt',
 '14375.txt',
 '73056.txt',
 '02315.txt',
 '43065.txt',
 '53682.txt',
 '84652.txt',
 '42638.txt',
 '78624.txt',
 '63028.txt',
 '68507.txt',
 '34051.txt',
 '89356.txt',
 '13692.txt',
 '68752.txt',
 '47031.txt',
 '57094.txt',
 '92051.txt',
 '47982.txt',
 '34057.txt',
 '52719.txt',
 '03847.txt',
 '56392.txt',
 '93085.txt',
 '48276.txt',
 '94216.txt',
 '70842.txt',
 '34857.txt',
 '39250.txt',
 '06482.txt',
 '60937.txt',
 '38156.txt',
 '18926.txt',
 '17236.txt',
 '40529.txt',
 '35189.txt',
 '19205.txt',
 '90756.txt',
 '35482.txt',
 '82704.txt',
 '70459.txt',
 '74602.txt',
 '21950.txt',
 '86039.txt',
 '48263.txt',
 '13058.txt',
 '80925.txt',
 '35861.txt',
 '25806.txt',
 '52960.txt',
 '83764.txt',
 '36974.txt',
 '27341.txt',
 '69524.txt',
 '14059.txt',
 '38465.txt',
 '98425.txt',
 '04917.txt',
 '95201.txt',
 '94860.txt',
 '56912.txt',
 '10547.txt',
 '34298.txt',
 '32560.txt',
 '91803.txt',
 '76035.txt',
 '59072.txt',
 '04762.txt',
 '54039.txt',
 '0261

In [17]:
import numpy
dictsfinal4 = {}
mylist1 = []
for textdata in os.listdir('img/labels/'):
    if textdata.endswith('.txt'):
        with open(os.path.join('img/labels/', textdata), encoding="UTF-8") as f:
            content = f.readlines()
            mylist1 = numpy.append(mylist1, content)
            f.close()

In [18]:
mylist1

array(['girl during sex: make me your slave me: muslim',
       'wants to stone gay people to death dresses like a drag queen',
       'what if punk is just gangsta rap for white kids', ...,
       'copsuckers be like the whole boot daddy',
       'i dont lift so that doesnt make me sexy? wallet and pay for all the shit you want. i bet its sexy when i lift my wallet and pay for all the shit you want',
       "fuck off!! you muslim bastards, we didn't win 2 wars just to hand england over to you!!!"],
      dtype='<U433')

In [19]:
missing = set(files_img)-set(files_txt)

In [20]:
missing

{'80613.png',
 '36781.png',
 '14037.png',
 '14097.png',
 '89607.png',
 '56974.png',
 '38720.png',
 '32695.png',
 '51687.png',
 '35796.png',
 '54367.png',
 '04397.png',
 '20718.png',
 '89640.png',
 '58069.png',
 '18937.png',
 '54629.png',
 '63578.png',
 '37815.png',
 '39256.png',
 '16759.png',
 '67183.png',
 '71823.png',
 '07634.png',
 '08743.png',
 '01268.png',
 '51376.png',
 '96578.png',
 '72431.png',
 '10285.png',
 '91627.png',
 '37965.png',
 '94176.png',
 '16947.png',
 '84097.png',
 '56830.png',
 '91764.png',
 '54823.png',
 '41680.png',
 '31470.png',
 '61537.png',
 '70948.png',
 '81359.png',
 '91602.png',
 '42167.png',
 '53469.png',
 '40982.png',
 '83607.png',
 '70614.png',
 '29164.png',
 '08624.png',
 '93512.png',
 '61792.png',
 '79613.png',
 '31684.png',
 '03947.png',
 '95812.png',
 '53419.png',
 '58321.png',
 '48271.png',
 '58096.png',
 '08671.png',
 '96710.png',
 '87596.png',
 '12694.png',
 '70936.png',
 '36982.png',
 '12936.png',
 '05864.png',
 '96423.png',
 '41502.png',
 '2549

In [21]:
original_dataset_dir = 'img/'
base_dataset_dir = 'img/data-final/'

In [22]:
original_label_path = original_dataset_dir + 'labels'
original_label_files = [f for f in listdir(original_label_path) if isfile(join(original_label_path, f))]

In [23]:
original_label_files

['36850.txt',
 '54028.txt',
 '04729.txt',
 '14375.txt',
 '73056.txt',
 '02315.txt',
 '43065.txt',
 '53682.txt',
 '84652.txt',
 '42638.txt',
 '78624.txt',
 '63028.txt',
 '68507.txt',
 '34051.txt',
 '89356.txt',
 '13692.txt',
 '68752.txt',
 '47031.txt',
 '57094.txt',
 '92051.txt',
 '47982.txt',
 '34057.txt',
 '52719.txt',
 '03847.txt',
 '56392.txt',
 '93085.txt',
 '48276.txt',
 '94216.txt',
 '70842.txt',
 '34857.txt',
 '39250.txt',
 '06482.txt',
 '60937.txt',
 '38156.txt',
 '18926.txt',
 '17236.txt',
 '40529.txt',
 '35189.txt',
 '19205.txt',
 '90756.txt',
 '35482.txt',
 '82704.txt',
 '70459.txt',
 '74602.txt',
 '21950.txt',
 '86039.txt',
 '48263.txt',
 '13058.txt',
 '80925.txt',
 '35861.txt',
 '25806.txt',
 '52960.txt',
 '83764.txt',
 '36974.txt',
 '27341.txt',
 '69524.txt',
 '14059.txt',
 '38465.txt',
 '98425.txt',
 '04917.txt',
 '95201.txt',
 '94860.txt',
 '56912.txt',
 '10547.txt',
 '34298.txt',
 '32560.txt',
 '91803.txt',
 '76035.txt',
 '59072.txt',
 '04762.txt',
 '54039.txt',
 '0261

In [24]:
original_image_path = original_dataset_dir + 'img'
original_image_files = [f for f in listdir(original_image_path) if isfile(join(original_image_path, f))]

In [25]:
original_image_files

['21753.png',
 '10582.png',
 '04615.png',
 '84316.png',
 '45109.png',
 '30642.png',
 '59231.png',
 '54397.png',
 '98047.png',
 '80614.png',
 '53296.png',
 '68721.png',
 '46895.png',
 '40216.png',
 '83921.png',
 '28160.png',
 '24769.png',
 '72940.png',
 '28976.png',
 '13902.png',
 '89302.png',
 '87023.png',
 '74059.png',
 '72893.png',
 '07463.png',
 '21650.png',
 '27856.png',
 '80941.png',
 '14965.png',
 '03942.png',
 '35680.png',
 '39854.png',
 '28134.png',
 '49271.png',
 '39256.png',
 '09347.png',
 '58419.png',
 '32674.png',
 '52691.png',
 '83429.png',
 '47290.png',
 '98305.png',
 '72654.png',
 '79568.png',
 '63418.png',
 '19372.png',
 '18306.png',
 '17493.png',
 '89503.png',
 '46387.png',
 '07261.png',
 '75160.png',
 '84016.png',
 '10867.png',
 '45269.png',
 '09128.png',
 '93610.png',
 '92016.png',
 '32915.png',
 '45827.png',
 '94617.png',
 '37298.png',
 '42681.png',
 '79162.png',
 '91768.png',
 '12678.png',
 '26598.png',
 '53976.png',
 '51973.png',
 '50462.png',
 '04397.png',
 '9702

In [26]:
print('number of images: {}, number of tags: {}'.format(len(original_image_files), len(original_label_files)))

number of images: 10000, number of tags: 9500


In [27]:
original_image_files = np.array(original_image_files)
original_label_files = np.array(original_label_files)

In [28]:
dataset_size = original_label_files.shape[0]
perm = np.arange(dataset_size)
np.random.shuffle(perm)
original_image_files = original_image_files[perm]
original_label_files = original_label_files[perm]

### setup a small dataset

In [29]:
if not os.path.isdir(base_dataset_dir):
    os.mkdir(base_dataset_dir)
    
small_label_path = os.path.join(base_dataset_dir, 'labels')
small_image_path = os.path.join(base_dataset_dir, 'thumbnails')
if not os.path.isdir(small_label_path):
    os.mkdir(small_label_path)
else:
    shutil.rmtree(small_label_path)
    os.mkdir(small_label_path)
    
if not os.path.isdir(small_image_path):
    os.mkdir(small_image_path)
else:
    shutil.rmtree(small_image_path)
    os.mkdir(small_image_path)

In [30]:
# copy label files
for fname in original_label_files[:9500]:
    src = os.path.join(original_label_path, fname)
    dst = os.path.join(small_label_path, fname)
    shutil.copyfile(src, dst)

In [31]:
# copy image files
for fname in original_label_files[:9500]:
    img_fname = fname[:-4]
    src = os.path.join(original_image_path, img_fname+".png")
    dst = os.path.join(small_image_path, img_fname+".png")
    shutil.copyfile(src, dst)

In [32]:
label_map = {'label_file' : [], 'word_tags' : []}
for fname in listdir(small_label_path): 
    f = join(small_label_path, fname)
    if isfile(f):
        f = open(f, encoding = "UTF-8")
        label_map['label_file'].append(fname)
        line = f.read().splitlines()
        label_map['word_tags'].append(line)
label_df = pd.DataFrame(label_map)
label_df.head()

,label_file,word_tags
0,36850.txt,[girl during sex: make me your slave me: muslim]
1,54028.txt,[wants to stone gay people to death dresses li...
2,04729.txt,[what if punk is just gangsta rap for white kids]
3,14375.txt,[wait a sec are u trying to gas me again]
4,73056.txt,[grandparents be like one little snack before ...


In [33]:
label_df.head()

,label_file,word_tags
0,36850.txt,[girl during sex: make me your slave me: muslim]
1,54028.txt,[wants to stone gay people to death dresses li...
2,04729.txt,[what if punk is just gangsta rap for white kids]
3,14375.txt,[wait a sec are u trying to gas me again]
4,73056.txt,[grandparents be like one little snack before ...


row[0].split("/")[1].split(".")[0]

In [34]:
train_df["label"]

0       0
1       0
2       0
3       0
4       0
       ..
8495    1
8496    1
8497    1
8498    1
8499    1
Name: label, Length: 8500, dtype: object

In [35]:
import os

file_path_img_sub = 'img/data-final/thumbnails'
file_path_text_sub = 'img/data-final/labels'

files_img1 = [f for f in os.listdir(file_path_img_sub) if f.endswith('.png')]
files_txt1 = [f for f in os.listdir(file_path_text_sub) if f.endswith('.txt')]

In [36]:
empty = []
for index, row in label_df.iterrows():
#     print(index,row[0].split(".")[0])
    empty.append(row[0].split(".")[0])
label_df["id"] = empty

In [37]:
label_df.head()

,label_file,word_tags,id
0,36850.txt,[girl during sex: make me your slave me: muslim],36850
1,54028.txt,[wants to stone gay people to death dresses li...,54028
2,04729.txt,[what if punk is just gangsta rap for white kids],04729
3,14375.txt,[wait a sec are u trying to gas me again],14375
4,73056.txt,[grandparents be like one little snack before ...,73056


In [38]:
train_df.head()

,img,label,text,id
0,img/42953.png,0,its their character not their color that matters,42953
1,img/23058.png,0,don't be afraid to love again everyone is not ...,23058
2,img/13894.png,0,putting bows on your pet,13894
3,img/37408.png,0,i love everything and everybody! except for sq...,37408
4,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403


In [39]:
dfinal = label_df.merge(train_df, how='inner')

In [40]:
dfinal.head()

,label_file,word_tags,id,img,label,text
0,36850.txt,[girl during sex: make me your slave me: muslim],36850,img/36850.png,1,girl during sex: make me your slave me: muslim
1,54028.txt,[wants to stone gay people to death dresses li...,54028,img/54028.png,1,wants to stone gay people to death dresses lik...
2,04729.txt,[what if punk is just gangsta rap for white kids],04729,img/04729.png,0,what if punk is just gangsta rap for white kids
3,73056.txt,[grandparents be like one little snack before ...,73056,img/73056.png,0,grandparents be like one little snack before y...
4,02315.txt,[don't forget me!!!!!! i'm an asshole too!!!],02315,img/02315.png,0,don't forget me!!!!!! i'm an asshole too!!!


In [41]:
label_df.head()

,label_file,word_tags,id
0,36850.txt,[girl during sex: make me your slave me: muslim],36850
1,54028.txt,[wants to stone gay people to death dresses li...,54028
2,04729.txt,[what if punk is just gangsta rap for white kids],04729
3,14375.txt,[wait a sec are u trying to gas me again],14375
4,73056.txt,[grandparents be like one little snack before ...,73056


In [42]:
from nltk.tokenize import word_tokenize
emptyv = []
for index, row in dfinal.iterrows():
#     print(word_tokenize(row[1][0]))
    emptyv.append(word_tokenize(row[1][0]))
dfinal["word_tags_new"] = emptyv

In [43]:
dfinal.head()

,label_file,word_tags,id,img,label,text,word_tags_new
0,36850.txt,[girl during sex: make me your slave me: muslim],36850,img/36850.png,1,girl during sex: make me your slave me: muslim,"[girl, during, sex, :, make, me, your, slave, ..."
1,54028.txt,[wants to stone gay people to death dresses li...,54028,img/54028.png,1,wants to stone gay people to death dresses lik...,"[wants, to, stone, gay, people, to, death, dre..."
2,04729.txt,[what if punk is just gangsta rap for white kids],04729,img/04729.png,0,what if punk is just gangsta rap for white kids,"[what, if, punk, is, just, gangsta, rap, for, ..."
3,73056.txt,[grandparents be like one little snack before ...,73056,img/73056.png,0,grandparents be like one little snack before y...,"[grandparents, be, like, one, little, snack, b..."
4,02315.txt,[don't forget me!!!!!! i'm an asshole too!!!],02315,img/02315.png,0,don't forget me!!!!!! i'm an asshole too!!!,"[do, n't, forget, me, !, !, !, !, !, !, i, 'm,..."


In [44]:
dfinal.pop('word_tags')

0        [girl during sex: make me your slave me: muslim]
1       [wants to stone gay people to death dresses li...
2       [what if punk is just gangsta rap for white kids]
3       [grandparents be like one little snack before ...
4           [don't forget me!!!!!! i'm an asshole too!!!]
                              ...                        
8495                               [ahmed i love you too]
8496    [photo of a survivor liberated from the auschw...
8497            [copsuckers be like the whole boot daddy]
8498    [i dont lift so that doesnt make me sexy? wall...
8499    [fuck off!! you muslim bastards, we didn't win...
Name: word_tags, Length: 8500, dtype: object

In [45]:
dfinal.pop('id')

0       36850
1       54028
2       04729
3       73056
4       02315
        ...  
8495    53427
8496    20693
8497    02935
8498    71325
8499    70561
Name: id, Length: 8500, dtype: object

In [46]:
dfinal.pop('img')

0       img/36850.png
1       img/54028.png
2       img/04729.png
3       img/73056.png
4       img/02315.png
            ...      
8495    img/53427.png
8496    img/20693.png
8497    img/02935.png
8498    img/71325.png
8499    img/70561.png
Name: img, Length: 8500, dtype: object

In [47]:
dfinal.pop('text')

0          girl during sex: make me your slave me: muslim
1       wants to stone gay people to death dresses lik...
2         what if punk is just gangsta rap for white kids
3       grandparents be like one little snack before y...
4             don't forget me!!!!!! i'm an asshole too!!!
                              ...                        
8495                                 ahmed i love you too
8496    photo of a survivor liberated from the auschwi...
8497              copsuckers be like the whole boot daddy
8498    i dont lift so that doesnt make me sexy? walle...
8499    fuck off!! you muslim bastards, we didn't win ...
Name: text, Length: 8500, dtype: object

In [48]:
dfinal.head()

,label_file,label,word_tags_new
0,36850.txt,1,"[girl, during, sex, :, make, me, your, slave, ..."
1,54028.txt,1,"[wants, to, stone, gay, people, to, death, dre..."
2,04729.txt,0,"[what, if, punk, is, just, gangsta, rap, for, ..."
3,73056.txt,0,"[grandparents, be, like, one, little, snack, b..."
4,02315.txt,0,"[do, n't, forget, me, !, !, !, !, !, !, i, 'm,..."


In [49]:
dfinal = dfinal.rename(columns={"word_tags_new": "word_tags"})

In [50]:
dfinal.shape

(8500, 3)

In [51]:
dfinal.head()

,label_file,label,word_tags
0,36850.txt,1,"[girl, during, sex, :, make, me, your, slave, ..."
1,54028.txt,1,"[wants, to, stone, gay, people, to, death, dre..."
2,04729.txt,0,"[what, if, punk, is, just, gangsta, rap, for, ..."
3,73056.txt,0,"[grandparents, be, like, one, little, snack, b..."
4,02315.txt,0,"[do, n't, forget, me, !, !, !, !, !, !, i, 'm,..."


In [52]:
dfinal['word_tags'].head()

0    [girl, during, sex, :, make, me, your, slave, ...
1    [wants, to, stone, gay, people, to, death, dre...
2    [what, if, punk, is, just, gangsta, rap, for, ...
3    [grandparents, be, like, one, little, snack, b...
4    [do, n't, forget, me, !, !, !, !, !, !, i, 'm,...
Name: word_tags, dtype: object

In [53]:
class Preprocessing(object):
    def __init__(self, data, source_column_name='word_tags', target_column_name='word_tags_clean'):
        self.data = data
        self.source_column_name = source_column_name
        self.feature_name = target_column_name
        
    def remove_punctuation(self, text):
        text_nopunct = "".join([char for char in text if char not in string.punctuation])# It will discard all punctuations
        return text_nopunct
    
    def tokenize(self, text):
        # Match one or more characters which are not word character
        tokens = re.split('\W+', text) 
        return tokens
    
    def remove_stopwords(self, tokenized_list):
        # Remove all English Stopwords
        stopword = nltk.corpus.stopwords.words('english')
        text = [word for word in tokenized_list if word not in stopword]
        return text
    
    def stemming(self, tokenized_text):
        ps = nltk.PorterStemmer()
        text = [ps.stem(word) for word in tokenized_text]
        return text
    
    def lemmatizing(self, tokenized_text):
        wn = nltk.WordNetLemmatizer()
        text = [wn.lemmatize(word) for word in tokenized_text]
        return text
    
    def alnum(self, text):
        def hasNumbers(inputString):
            return any(char.isdigit() for char in inputString)
        
        text = [word for word in text if len(word) > 0 and word.isalnum() and not word.isdigit() and not hasNumbers(word)]
        return text
    
    def tokens_to_string(self, tokens_string):
        try:
            list_obj = ast.literal_eval(tokens_string)
            text = " ".join(list_obj)
        except:
            text = None
        return text
    
    def dropna(self):
        feature_name = self.feature_name
        if self.data[feature_name].isnull().sum() > 0:
            column_list=[feature_name]
            self.data = self.data.dropna(subset=column_list)
            return self.data
        
    def preprocessing(self):
        nopunc = self.source_column_name + '_nopunc'
        tokenized = self.source_column_name + '_tokenized'
        nostop = self.source_column_name + '_nostop'
        stemmed = self.source_column_name + '_stemmed'
        lemmatized = self.source_column_name + '_lemmatized'
        self.data[nostop] = self.data[self.source_column_name].apply(lambda x: self.remove_stopwords(x))
        self.data[self.feature_name] = self.data[nostop].apply(lambda x: self.alnum(x))  
        return self.data
    
    def save(self, filepath="tags_cleaned.csv"):
        self.data.to_csv(filepath, index=False, sep=',')  
        
    def load(self, filepath="tags_cleaned.csv"):
        self.data = pd.read_csv(filepath)
        return self.data

In [54]:
pp = Preprocessing(dfinal.copy(), source_column_name='word_tags', target_column_name='word_tags_clean')
label_df_clean = pp.preprocessing()

In [55]:
label_df_clean.head()

,label_file,label,word_tags,word_tags_nostop,word_tags_clean
0,36850.txt,1,"[girl, during, sex, :, make, me, your, slave, ...","[girl, sex, :, make, slave, :, muslim]","[girl, sex, make, slave, muslim]"
1,54028.txt,1,"[wants, to, stone, gay, people, to, death, dre...","[wants, stone, gay, people, death, dresses, li...","[wants, stone, gay, people, death, dresses, li..."
2,04729.txt,0,"[what, if, punk, is, just, gangsta, rap, for, ...","[punk, gangsta, rap, white, kids]","[punk, gangsta, rap, white, kids]"
3,73056.txt,0,"[grandparents, be, like, one, little, snack, b...","[grandparents, like, one, little, snack, go, h...","[grandparents, like, one, little, snack, go, h..."
4,02315.txt,0,"[do, n't, forget, me, !, !, !, !, !, !, i, 'm,...","[n't, forget, !, !, !, !, !, !, 'm, asshole, !...","[forget, asshole]"


In [56]:
non_empty_tags = label_df_clean['word_tags_clean'].apply(lambda x: len(x) > 0)
label_df_clean = label_df_clean[non_empty_tags]

In [57]:
label_df_clean.head()

,label_file,label,word_tags,word_tags_nostop,word_tags_clean
0,36850.txt,1,"[girl, during, sex, :, make, me, your, slave, ...","[girl, sex, :, make, slave, :, muslim]","[girl, sex, make, slave, muslim]"
1,54028.txt,1,"[wants, to, stone, gay, people, to, death, dre...","[wants, stone, gay, people, death, dresses, li...","[wants, stone, gay, people, death, dresses, li..."
2,04729.txt,0,"[what, if, punk, is, just, gangsta, rap, for, ...","[punk, gangsta, rap, white, kids]","[punk, gangsta, rap, white, kids]"
3,73056.txt,0,"[grandparents, be, like, one, little, snack, b...","[grandparents, like, one, little, snack, go, h...","[grandparents, like, one, little, snack, go, h..."
4,02315.txt,0,"[do, n't, forget, me, !, !, !, !, !, !, i, 'm,...","[n't, forget, !, !, !, !, !, !, 'm, asshole, !...","[forget, asshole]"


In [58]:
label_df_clean.shape

(8488, 5)

In [59]:
num_of_samples = label_df_clean.shape[0]
num_of_samples

8488

## Glove

In [60]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
training_samples = num_of_samples
tag_vocabulary_size = 10000
max_words = tag_vocabulary_size

In [61]:
glove_dir = 'glove/glove.6B/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = "UTF-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [62]:
tokenizer = Tokenizer(num_words=max_words)
texts = []
for tag_list in label_df_clean['word_tags']:
    texts.append(' '.join(tag_list))
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found {} unique tokens'.format(len(word_index)))
tag_data = pad_sequences(sequences, maxlen=maxlen)

Found 8921 unique tokens


In [63]:
tag_data.shape

(8488, 100)

In [64]:
tag_data

array([[   0,    0,    0, ...,  768,   21,   61],
       [   0,    0,    0, ...,    3, 2780, 1940],
       [   0,    0,    0, ...,   16,   43,  124],
       ...,
       [   0,    0,    0, ...,  652, 8921,  810],
       [   0,    0,    0, ...,   77,    1,   78],
       [   0,    0,    0, ...,  148,    5,    1]], dtype=int32)

In [65]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [66]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.49886   ,  0.76602   ,  0.89750999, ..., -0.41179001,
         0.40538999,  0.78504002],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [67]:
tag_data.shape

(8488, 100)

In [68]:
tag_input = Input(shape=(None,), dtype='int32', name='tag')
embedded_tag = layers.Embedding(max_words, embedding_dim)(tag_input)
encoded_tag = layers.LSTM(512)(embedded_tag)

## CONV2D

In [69]:
from keras.applications import VGG16

image_input = Input(shape=(150, 150, 3), name='image')
vgg16 = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))(image_input)
x = layers.Flatten()(vgg16) 
x = layers.Dense(256, activation='relu')(x)

In [70]:
import tensorflow

concatenated = layers.concatenate([x, encoded_tag], axis=-1)
output = layers.Dense(1, activation='sigmoid')(concatenated)

model = Model([image_input, tag_input], output)

# model.layers[1].trainable = False # freeze VGG16 convolutional base
model.layers[4].set_weights([embedding_matrix]) 
# model.layers[4].trainable = False # freeze GloVe word embedding

class new_callback(tensorflow.keras.callbacks.Callback):
    def epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy')> 0.65): # select the accuracy
            print("\n !!! 65% accuracy, no further training !!!")
            self.model.stop_training = True
            
callbacks = new_callback()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [71]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 4, 4, 512)    14714688    image[0][0]                      
__________________________________________________________________________________________________
tag (InputLayer)                (None, None)         0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 8192)         0           vgg16[1][0]                      
____________________________________________________________________________________________

In [72]:
# # model.layers[1].trainable = False # freeze VGG16 convolutional base
# model.layers[4].set_weights([embedding_matrix])
# model.layers[4].trainable = False # freeze GloVe word embedding

In [73]:
label_df_clean.head()

,label_file,label,word_tags,word_tags_nostop,word_tags_clean
0,36850.txt,1,"[girl, during, sex, :, make, me, your, slave, ...","[girl, sex, :, make, slave, :, muslim]","[girl, sex, make, slave, muslim]"
1,54028.txt,1,"[wants, to, stone, gay, people, to, death, dre...","[wants, stone, gay, people, death, dresses, li...","[wants, stone, gay, people, death, dresses, li..."
2,04729.txt,0,"[what, if, punk, is, just, gangsta, rap, for, ...","[punk, gangsta, rap, white, kids]","[punk, gangsta, rap, white, kids]"
3,73056.txt,0,"[grandparents, be, like, one, little, snack, b...","[grandparents, like, one, little, snack, go, h...","[grandparents, like, one, little, snack, go, h..."
4,02315.txt,0,"[do, n't, forget, me, !, !, !, !, !, !, i, 'm,...","[n't, forget, !, !, !, !, !, !, 'm, asshole, !...","[forget, asshole]"


In [74]:
dim = (150, 150)
X_image_train = []
X_tag_train = tag_data
y_train = label_df_clean["label"]
    
for fname in listdir(small_image_path):
    fpath = os.path.join(small_image_path, fname)
    im = cv2.imread(fpath)
    im_resized = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    X_image_train.append(im_resized)
#     y_train.append(1)
    
# # add wrong tag samples
# num_negative_samples = len(y_train)
# for i in range(num_negative_samples):
#     image = X_image_train[i]
#     X_image_train.append(image)
#     j = (i + 1) % num_negative_samples # get a different tag
#     tag = X_tag_train[j]
#     X_tag_train = np.append(X_tag_train, tag) 
#     y_train
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()
# le.fit(y_train)
# y_train = le.transform(y_train)

In [75]:
X_image_train = np.array(X_image_train)
X_tag_train   = np.array(tag_data)
y_train       = np.array(y_train)

In [76]:
perm = np.arange(y_train.shape[0])
np.random.shuffle(perm)
X_image_train = X_image_train[perm]
X_tag_train   = X_tag_train[perm]
y_train       = y_train[perm]

In [77]:
X_image_train.shape, X_tag_train.shape, y_train.shape

((8488, 150, 150, 3), (8488, 100), (8488,))

In [78]:
X_image_train

array([[[[ 57,  57,  57],
         [106, 106, 106],
         [104, 104, 104],
         ...,
         [150, 150, 150],
         [139, 139, 139],
         [143, 143, 143]],

        [[ 35,  35,  35],
         [ 66,  66,  66],
         [ 61,  61,  61],
         ...,
         [ 88,  88,  88],
         [ 86,  86,  86],
         [ 72,  72,  72]],

        [[ 35,  35,  35],
         [ 67,  67,  67],
         [ 77,  77,  77],
         ...,
         [ 76,  76,  76],
         [ 75,  75,  75],
         [ 85,  85,  85]],

        ...,

        [[142, 142, 142],
         [142, 142, 142],
         [138, 138, 138],
         ...,
         [222, 222, 222],
         [220, 220, 220],
         [220, 220, 220]],

        [[140, 140, 140],
         [139, 139, 139],
         [138, 138, 138],
         ...,
         [222, 222, 222],
         [219, 219, 219],
         [210, 210, 210]],

        [[139, 139, 139],
         [134, 134, 134],
         [133, 133, 133],
         ...,
         [218, 218, 218],
        

In [79]:
X_tag_train

array([[   0,    0,    0, ...,    0,  112, 4256],
       [   0,    0,    0, ...,    5,   27, 8371],
       [   0,    0,    0, ...,  757,   16, 2045],
       ...,
       [   0,    0,    0, ..., 1216, 6471, 1895],
       [   0,    0,    0, ...,   15, 3127,  976],
       [   0,    0,    0, ...,    9,    2, 5583]], dtype=int32)

In [80]:
y_train

array(['0', '0', '0', ..., '1', '0', '1'], dtype=object)

In [81]:
# from keras.callbacks import EarlyStopping
# callback = EarlyStopping(monitor='acc', patience=0)
# #,min_delta=0.005, baseline=0.,verbose=1

In [ ]:
%%timeit
# batch_size = 64
# nb_epoch = 5
model.fit([X_image_train, X_tag_train],y_train, batch_size = 64, epochs = 10, callbacks = [callbacks])

/home/vinod/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
8488/8488 [==============================] - 1461s 172ms/step - loss: 894.7423 - accuracy: 0.6291
Epoch 2/10
8256/8488 [============================>.] - ETA: 40s - loss: 0.9488 - accuracy: 0.6818

In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("multi-input-models-epoch-30.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("multi-input-models-epoch-30.h5")
print("Saved model to disk")
 
# load json and create model
json_file = open('multi-input-models-epoch-30.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("multi-input-models-epoch-30.h5")
print("Loaded model from disk")